## Expanding the CIP netCDFfiles and adding data from nav files

Status 'Turn into script'
- imports OK
- nc_save_with_check OK
- get_safire_flightid OK


TODO: 
- check and updata floor_to_sec_res
- check and update standardize_cip_netcdf (handling of nav-files and metadata remains)
- separate paths and struct information for reuse

In [8]:
import xarray as xr # read netcdf-files
import glob # allows for wildcards in filemanagement
import numpy as np
import os #get a list of all directories/files
import re #regex

from datetime import date
from utils.func_nc import floor_to_sec_res
from utils.nc_utils import nc_save_with_check, standardize_cip_netcdf
from utils.flight_utils import get_safire_flightid

In [9]:
# sample rate to process (current possibilities: 1,5,12 sek)
sample_rate = 5

In [ ]:
# -- Paths to datafiles
nav_main_path = '/home/ninalar/Documents/MC2/2022-islas/' # Local disk path to nav data:
cip_main_path = f'/home/ninalar/Documents/MC2/Results_2022-islas/{sample_rate}sAveraging/'# Local disk path to SODA processed CIP files
save_path = '/home/ninalar/Documents/MC2/Results_2022-islas/Processed/CIP_processed/' # Save file path

# structure of file names (for access)
file_struct = {'cip':'/*CIP.nc',
               'nav_tdyn':'/*_TDYN_*.nc',
               'nav_nav': '/*_NAV_*.nc'}

flights, safire_to_islas = get_safire_flightid(nav_main_path)

In [ ]:
# Process all flights in the folders
for flight in flights:
    print(flight)

    # get navigation filenames
    nav_tdyn_file = glob.glob(nav_main_path + flight + file_struct['nav_tdyn']) 
    nav_nav_file = glob.glob(nav_main_path + flight + file_struct['nav_nav'])

    # ---- Get CDP and NAV data from flight
    if isinstance(safire_to_islas[flight],list):
        # if more than one islasid for the safireid
        
        # flights with more than one islasids and must be addressed separately:
        for islasid in safire_to_islas[flight]:
            cip_file = glob.glob(cip_main_path + flight + f'/{islasid}' + file_struct['cip']) 
            print(f'Reading: {cip_file[0]}, {nav_nav_file[0]} and {nav_tdyn_file[0]}')
            cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_tdyn_file[0],nav_nav_file[0], islasid) # add islasid and coordinates to xds

            # save to new netCDF
            savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'
            nc_save_with_check(savefile, cip_updated_xds)
    else:
        cip_file = glob.glob(cip_main_path + flight + file_struct['cip'])
        print(f'Reading: {cip_file[0]}, {nav_nav_file[0]} and {nav_tdyn_file[0]}')
        islasid = safire_to_islas[flight]
        cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_tdyn_file[0],nav_nav_file[0], islasid) # add islasid and coordinates to xds

        #Save to new netCDF
        savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'      
        nc_save_with_check(savefile, cip_updated_xds)

as220014
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220014/04032022_071513_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220014_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as220014_L1_V1.nc
Exiting...
as220009
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220009/03262022_080110_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220009_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220009_L1_V1.nc
Exiting...
as220010
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220010/03262022_145034_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220010_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220010_